
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session.                                                                                                 |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0).                               |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |  Changes the session type to Glue ETL.                                                                                                                    |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X.                                                                           |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer.                      |

In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import requests as re
import pandas as pd
from datetime import datetime
import numpy as np

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::642035821867:role/AdministratorAccessforDemo
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: b651b5da-4b5f-4411-b632-d32847db7417
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session b651b5da-4b5f-4411-b632-d32847db7417 to get into ready status...
Session b651b5da-4b5f-4411-b632-d32847db7417 has been created.



In [2]:
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [3]:
#Current date for file reading
today = datetime.today().strftime('%Y-%m-%d')

In [4]:
#Load in pr data current 0
pr_current = pd.read_fwf("s3://bls-timeseries-data/pr.data.0.Current/" + today + "/data.txt")
#Load in pop data
pop_data = pd.read_json("s3://data-usa/pop_data.json")

In [8]:
#Clean pr current values
pr_current['value'] = pr_current['value'].str.replace("\tR", "").astype(float)

In [9]:
#Calculate the mean & std dev of the us population from 2013 - 2018 inclusive
pop_mean = np.mean(pop_data['Population'])
pop_std_dev = np.std(pop_data['Population'])

print("Population mean = " + str(pop_mean))
print("Population Standard Deviation = " + str(pop_std_dev))

Population mean = 319486425.125
Population Standard Deviation = 4913388.652424171


In [10]:
#Create temp tables - easier to use sql for this
spark.createDataFrame(pr_current).createOrReplaceTempView("pr_curr")
spark.createDataFrame(pop_data).createOrReplaceTempView("pop_data")

In [11]:
max_value = spark.sql("select series_id, max(year) as highest_year, max(value) as top_value from pr_curr group by series_id")
max_value.show()

+-----------+------------+---------+
|  series_id|highest_year|top_value|
+-----------+------------+---------+
|PRS85006102|        2022|     25.8|
|PRS31006023|        2022|    9.954|
|PRS32006111|        2022|      9.6|
|PRS84006162|        2022|     15.0|
|PRS30006152|        2022|     34.6|
|PRS88003041|        2022|     17.1|
|PRS88003063|        2022|    9.947|
|PRS85006173|        2022|    9.991|
|PRS30006033|        2022|    9.915|
|PRS32006082|        2020|     16.8|
|PRS30006082|        2020|     16.4|
|PRS32006011|        2022|      5.6|
|PRS84006161|        2022|      6.5|
|PRS32006162|        2022|     12.9|
|PRS88003013|        2022|    9.997|
|PRS30006013|        2022|    9.912|
|PRS88003033|        2022|    9.907|
|PRS88003082|        2022|     65.5|
|PRS88003083|        2022|    9.996|
|PRS31006131|        2020|      6.8|
+-----------+------------+---------+
only showing top 20 rows


In [12]:
#Check to make sure number of records matches sql query
spark.sql("select series_id, max(year) as highest_year, max(value) as top_value from pr_curr group by series_id").count()

282


In [13]:
#Check cont.
pr_current['series_id'].drop_duplicates().count()

282


In [14]:
#Join pop data to pr current data and filter for specific value
joined_dfs_for_report = spark.sql("select a.series_id, \
          a.year, \
          a.period, \
          a.value, \
          b.Population \
          from pr_curr a \
          left join pop_data b on a.year = b.Year \
          where a.series_id = 'PRS30006032' and a.year = '2018' and a.period = 'Q01'")

joined_dfs_for_report.show()

+-----------+----+------+-----+----------+
|  series_id|year|period|value|Population|
+-----------+----+------+-----+----------+
|PRS30006032|2018|   Q01| -0.2| 322903030|
+-----------+----+------+-----+----------+


In [15]:
#making sure value is correct
spark.sql("select a.series_id, \
          a.year, \
          a.period, \
          a.value, \
          b.Population \
          from pr_curr a \
          left join pop_data b on a.year = b.Year \
          where a.series_id = 'PRS30006032' and a.year = '2018'").show()

+-----------+----+------+-----+----------+
|  series_id|year|period|value|Population|
+-----------+----+------+-----+----------+
|PRS30006032|2018|   Q01| -0.2| 322903030|
|PRS30006032|2018|   Q02|  0.4| 322903030|
|PRS30006032|2018|   Q03|  5.2| 322903030|
|PRS30006032|2018|   Q04|  0.5| 322903030|
|PRS30006032|2018|   Q05|  1.8| 322903030|
+-----------+----+------+-----+----------+
